Win conditions of tik tac toe are if any straight, diagonal, or horizontal are all the same thing. I can create of set of each win-list from the beginning, and update it. The win list will include each horizontal, each straight, and each diagonal (o lit, there are only two diagonals). If an X and an O exist on the same list, that win list is now impossible. <br>
The board makes most sense as a 2d array.

In [1]:
import pandas as pd
import numpy as np
import random
import time

In [2]:
def board_creator(x):
    board = []
    for i in range(x):
        board.append(['E']*x)
    return board

In [3]:
board_creator(5)

[['E', 'E', 'E', 'E', 'E'],
 ['E', 'E', 'E', 'E', 'E'],
 ['E', 'E', 'E', 'E', 'E'],
 ['E', 'E', 'E', 'E', 'E'],
 ['E', 'E', 'E', 'E', 'E']]

A horizontal win, 0,0, 0,1, 0,2 : the pattern is for each y of same x value.
A vertical win, 0,0, 1,0, 2,0
A diagonal win, 0,0 ,1,1, 2,2

Start with just horizontal win. Do horizontal win, vertical win, then diagonal win.

In [4]:
def to_state(board):
    bl = sum(board,[]) #board as 1d list
    bc = 0 #board code, unique number created from board
    for i in range(len(bl)):
        m = 0 #multiplier
        if bl[i] == 'E':
            m = 1
        elif bl[i] == 'X':
            m = 2
        else: #maybe check only these 3 things happen
            m = 3
        bc += m*10**i
    return bc

In [5]:
def get_actions(board): #aka get legal moves
    bl = sum(board,[])
    actions = [i for i, x in enumerate(bl) if x == 'E']
    #print(actions)
    return actions

In [6]:
def get_next_states(board,turn,board_too=False):
    pos_actions = get_actions(board)
    #print('pos_actions')
    #print(pos_actions)
    next_states = []
    next_boards = []
    next_moves = []
    for action in pos_actions:
        b2 = [x[:] for x in board] #b2 is board copy, worry ab fastest implementation of this later
        x = action % len(board)
        y = int(action/len(board))
        #print('x, y')
        #print(x,y)
        #print(x,y,b2[y][x])
        b2[y][x] = turn #why did i write this?
        next_states.append(to_state(b2))
        next_boards.append(b2)
        next_moves.append([x,y])
#     print('***possible boards***')
#     for b in next_boards:
#         print_board(b)
#     print('********************')
    if board_too:
        return next_states, next_boards, next_moves
    return next_states

In [7]:
#returns a random state that shares the highest value when minmax chosen from policy
#returns any random state when minmax not chosen from policy

def choose_next_move(board,turn,policy,states):
    highest = -100 #will always be lower than any state value since -100 < avg([-1,-1,-1,..])

    values = [] #list of values of each next state
    
    next_states, next_boards, next_moves = get_next_states(board,turn,board_too=True)
#     print('next boards')
#     for b in next_boards:
#         print_board(b)
#     print('*******')
    #print(next_states)
    #Real quick, return something if in random part of policy
    if random.random() > policy:
        r = random.randint(0,len(next_states)-1)
        return next_states[r], next_boards[r], next_moves[r]
    
    #actual beginning of function
    for i in range(len(next_states)):
        if next_states[i] in list(states.keys()): #if it's in state.keys it'll have a value
            v = states[next_states[i]]['value']
            values.append(v)
            if v > highest:
                highest = v
            #print('poop')
        else:
            values.append(0)
            if 0 > highest:
                highest = 0
            #print('pee')
    #print([i for i, x in enumerate(values) if x == highest])
    next_index = random.choice([i for i, x in enumerate(values) if x == highest])
    #print(next_states[next_index], next_boards[next_index], next_moves[next_index])
    return next_states[next_index], next_boards[next_index], next_moves[next_index]

In [8]:
def print_board(board):
    print(pd.DataFrame(board))
#     print('current board state')
#     print(to_state(board))
#     print('legal actions')
#     print(get_actions(board))
#     print('next states')
#     print(get_next_states(board,p))
#     print('seen states')
#     print(states)

In [9]:
def get_turn(turns):
    if turns %2 == 0:
        return 'O'
    else:
        return 'X'

In [10]:
def win_reward(p,who_won):
    if p == who_won:
        return 1
    elif who_won == 'T':
        return 0
    else:
        return -1

This functions plays CPU players against each other to get better and outputs a model containing state value functions for each visited state in the training process. The policy is a parameter, however it is set to 1 (argmax) for the final 10% of episodes for code quality check.

In [11]:
def train(board_size,episodes,policy):
    #states = {} #seen states
                #need to contain a list of all rewards
                #avg of all rewards (0 if not)
    x_states = {} #X goes first always (by stipulation)
    o_states = {}
    tie_count = []
    x_win_count = []
    o_win_count = []
    t = time.time()
    for i in range(episodes):
#         print('Enter Board Size: ')
#         size = int(input())
        print_all = False
        if i % (episodes/10) == 0:
            print([time.time()-t, i])
            t = time.time()
            print_all = True
        board = board_creator(board_size)
        
        policy = policy
        if i > episodes * 0.9:
            policy =1
        #reset and setup win conditions. 
        #Can prob make this faster by not having to reset this every time
        horizontal_lists = {}
        vertical_lists = {}
        diag_1 = {}
        diag_2 = {}
        for i in range(board_size):
            horizontal_lists[i] = {'X':0, 'O':0}
            vertical_lists[i] = {'X': 0, 'O': 0}
        x_diag_1 = 0
        x_diag_2 = 0
        o_diag_1 = 0
        o_diag_2 = 0
        turns = board_size**2 #editted from just size, the input one
        tie_flag = True

        state_trace_x = [] 
        state_trace_o = []
        
        while(turns > 0): 
            
            #list of states seen in the episode, not the whole thing, used to go back and update rewards

            
            who_win = 'E' 
        
            p = get_turn(turns)
            
            
            e_count = 0
            for i in board:
                for k in i:
                    if k=='E':
                        e_count+=1
                        
            if e_count != turns:
                print('ilegal game')
            
            if print_all:
                print_board(board)
                #print(vertical_lists)
            next_state, next_board, next_move = 0,0,0
            if p == 'X':
                next_state, next_board, next_move = choose_next_move(board,p,policy,x_states)
            else:
                next_state, next_board, next_move = choose_next_move(board,p,policy,o_states)
            #dont actually need next_board...or even next_state
            if p == 'X':
                state_trace_x.append(next_state)
            else:
                state_trace_o.append(next_state)
        
    
            #Update win lists (which are used to check if a win has occured)
            x,y = next_move[0], next_move[1]
            horizontal_lists[x][p] += 1
            vertical_lists[y][p] += 1
            if (x == y):
                if (p =='X'):
                    x_diag_1 += 1
                else:
                    o_diag_1 += 1
            if ((x+y) == board_size-1):
                if (p =='X'):
                    x_diag_2 += 1
                else:
                    o_diag_2 += 1
                    
            #update board
            board[y][x] = p
            
            
            if (horizontal_lists[x][p] == board_size or x_diag_1 == board_size or x_diag_2 == board_size or o_diag_1 == board_size or o_diag_2 == board_size):
                tie_flag = False
                who_win = p
                if print_all:
                    print(x,y)
                    print_board(board)
                    print(p + ' wins!')
                break
            if (vertical_lists[y][p] == board_size):
                tie_flag = False
                who_win = p
                if print_all:
                    print(x,y)
                    print_board(board)
                    print(p + ' wins!')

                break
            turns -= 1

        #print(pd.DataFrame(board))
        if (tie_flag):
            who_win = 'T'
            tie_count.append(1)
        else:
            tie_count.append(0)
        
        if who_win == 'X':
            x_win_count.append(1)
        else:
            x_win_count.append(0)
        
        if who_win == 'O':
            o_win_count.append(1)
        else:
            o_win_count.append(0)
            
        
        
        #update state reward values and value history
        reward = 0
        for i in range(len(state_trace_x)):
            c_state = state_trace_x[-(i+1)] #chosen state
            #instantiate state in dict if never seen. Maybe can do faster
            if c_state not in list(x_states.keys()):
                x_states[c_state] = {'denomenator': 0, 'value':0}
            if i == 0: #final state, since c_state reverses list, the first iteration will be the final state
                reward = win_reward('X',who_win)
#                 x_states[c_state]['rewards'].append(reward)
#                 x_states[c_state]['value'] = np.mean(x_states[c_state]['rewards'])
                
                x_states[c_state]['value'] = reward
                x_states[c_state]['denomenator'] = 1
                
                continue
            local_reward = 0 #just to show how this'd go if each move before win had a reward
            gamma = 0.9
            reward *= gamma #get discount of reward of future actions
#             x_states[c_state]['rewards'].append(local_reward + reward)
#             x_states[c_state]['value'] = np.mean(x_states[c_state]['rewards']) #CAN BE FASTER
            x_states[c_state]['denomenator'] +=1
            x_states[c_state]['value'] = (x_states[c_state]['value'] + reward)/ x_states[c_state]['denomenator']
        
        reward = 0
        for i in range(len(state_trace_o)):
            c_state = state_trace_o[-(i+1)]
            #instantiate state in dict if never seen. Maybe can do faster
            if c_state not in list(o_states.keys()):
                o_states[c_state] = {'denomenator': 0, 'value':0}
            if i == 0: #final state, since c_state reverses list, the first iteration will be the final state
                reward = win_reward('O',who_win)
                o_states[c_state]['value'] = reward
                o_states[c_state]['denomenator'] = 1
                continue
            local_reward = 0 #just to show how this'd go if each move before win had a reward
            gamma = 0.9
            reward *= gamma #get discount of reward of future actions
            o_states[c_state]['denomenator'] +=1
            o_states[c_state]['value'] = (o_states[c_state]['value'] + reward)/ o_states[c_state]['denomenator']
        
        

    return x_states,o_states, tie_count, x_win_count,o_win_count

In [12]:
x_states,o_states,tc,x_wins,o_wins = train(3,100000,0.83)

[0.0, 0]
   0  1  2
0  E  E  E
1  E  E  E
2  E  E  E
   0  1  2
0  E  E  X
1  E  E  E
2  E  E  E
   0  1  2
0  E  E  X
1  O  E  E
2  E  E  E
   0  1  2
0  E  X  X
1  O  E  E
2  E  E  E
   0  1  2
0  E  X  X
1  O  O  E
2  E  E  E
   0  1  2
0  E  X  X
1  O  O  E
2  E  X  E
   0  1  2
0  O  X  X
1  O  O  E
2  E  X  E
   0  1  2
0  O  X  X
1  O  O  E
2  E  X  X
2 1
   0  1  2
0  O  X  X
1  O  O  O
2  E  X  X
O wins!
[13.350221872329712, 10000]
   0  1  2
0  E  E  E
1  E  E  E
2  E  E  E
   0  1  2
0  E  X  E
1  E  E  E
2  E  E  E
   0  1  2
0  E  X  E
1  E  E  E
2  E  E  O
   0  1  2
0  E  X  E
1  E  E  E
2  E  X  O
   0  1  2
0  E  X  E
1  E  O  E
2  E  X  O
   0  1  2
0  E  X  E
1  E  O  X
2  E  X  O
   0  1  2
0  E  X  E
1  O  O  X
2  E  X  O
   0  1  2
0  E  X  X
1  O  O  X
2  E  X  O
0 0
   0  1  2
0  O  X  X
1  O  O  X
2  E  X  O
O wins!
[13.290833234786987, 20000]
   0  1  2
0  E  E  E
1  E  E  E
2  E  E  E
   0  1  2
0  E  E  E
1  E  E  E
2  E  X  E
   0  1  2
0  E  E  O
1  E  E  

In [15]:
#number of unique visited states
print(len(list(x_states.keys())))
print(len(list(o_states.keys())))

2689
2704


Code below generates a story about what happened during the training (how many times X won, O won, or there was a tie, for each decile of episodes)

In [18]:
x_by_10 = []
o_by_10 = []
tie_by_10 = []
for i in range(10):
    t,v = int(i*len(x_wins)/10), int((i+1)*len(x_wins)/10)
    try:
        x_by_10.append(pd.Series(x_wins).iloc[t:v].value_counts()[1])
    except:
        x_by_10.append(0)
    try:
        o_by_10.append(pd.Series(o_wins).iloc[t:v].value_counts()[1])
    except:
        o_by_10.append(0)
    try:
        tie_by_10.append(pd.Series(tc).iloc[t:v].value_counts()[1])
    except:
        tie_by_10.append(0)
print(x_by_10)
print(o_by_10)
print(tie_by_10)

[4788, 4373, 4566, 4676, 4440, 4571, 4505, 4476, 4726, 9997]
[1433, 1238, 1204, 1121, 1112, 1126, 1147, 1110, 1131, 0]
[3779, 4389, 4230, 4203, 4448, 4303, 4348, 4414, 4143, 3]


Function below lets one play against a model (for now it only lets you play as O, 2nd)

In [21]:
def play(board_size,model):

    for i in range(1):
#         print('Enter Board Size: ')
#         size = int(input())
        

        board = board_creator(board_size)
        
        horizontal_lists = {}
        vertical_lists = {}
        diag_1 = {}
        diag_2 = {}
        for i in range(board_size):
            horizontal_lists[i] = {'X':0, 'O':0}
            vertical_lists[i] = {'X': 0, 'O': 0}
        x_diag_1 = 0
        x_diag_2 = 0
        o_diag_1 = 0
        o_diag_2 = 0
        turns = board_size**2 #editted from just size, the input one
        tie_flag = True

        
        
        
        
        while(turns > 0): 
            
            #list of states seen in the episode, not the whole thing, used to go back and update rewards
            print_board(board)
            
            who_win = 'E' 

            e_count = 0
            for i in board:
                for k in i:
                    if k=='E':
                        e_count+=1
            if e_count != turns:
                print('illegal game')
                
            p = get_turn(turns)
            print(p)
            next_state, next_board, next_move = 0,0,0
            if p == 'X':
                next_state, next_board, next_move = choose_next_move(board,p,2,model)
                print('x')
            else:
                i_1= int(input('Y coordinate'))
                i_2= int(input('X coordinate'))
                next_move = [i_2,i_1]
            
        
    
            #Update win lists (which are used to check if a win has occured)
            x,y = next_move[0], next_move[1]

            horizontal_lists[x][p] += 1
            vertical_lists[y][p] += 1
            if (x == y):
                if (p =='X'):
                    x_diag_1 += 1
                else:
                    o_diag_1 += 1
            if ((x+y) == board_size-1):
                if (p =='X'):
                    x_diag_2 += 1
                else:
                    o_diag_2 += 1
                    
            #update board
            print(p + ' move')
            print(vertical_lists)
            board[y][x] = p
            e_count = 0
            for i in board:
                for k in i:
                    if k=='E':
                        e_count+=1
            if e_count != turns -1:
                print('illegal game')
            
            if (horizontal_lists[x][p] == board_size or x_diag_1 == board_size or x_diag_2 == board_size or o_diag_1 == board_size or o_diag_2 == board_size):
                print(p + ' wins!') 
                print_board(board)
                tie_flag = False
                who_win = p
                break
            if (vertical_lists[y][p] == board_size):
                print(p + ' wins!')
                print_board(board) 
                tie_flag = False
                who_win = p
                break
            turns -= 1
        print_board(board)

In [24]:
play(3,x_states)

   0  1  2
0  E  E  E
1  E  E  E
2  E  E  E
X
x
X move
{0: {'X': 1, 'O': 0}, 1: {'X': 0, 'O': 0}, 2: {'X': 0, 'O': 0}}
   0  1  2
0  E  E  X
1  E  E  E
2  E  E  E
O
Y coordinate1
X coordinate1
O move
{0: {'X': 1, 'O': 0}, 1: {'X': 0, 'O': 1}, 2: {'X': 0, 'O': 0}}
   0  1  2
0  E  E  X
1  E  O  E
2  E  E  E
X
x
X move
{0: {'X': 1, 'O': 0}, 1: {'X': 1, 'O': 1}, 2: {'X': 0, 'O': 0}}
   0  1  2
0  E  E  X
1  X  O  E
2  E  E  E
O
Y coordinate2
X coordinate1
O move
{0: {'X': 1, 'O': 0}, 1: {'X': 1, 'O': 1}, 2: {'X': 0, 'O': 1}}
   0  1  2
0  E  E  X
1  X  O  E
2  E  O  E
X
x
X move
{0: {'X': 2, 'O': 0}, 1: {'X': 1, 'O': 1}, 2: {'X': 0, 'O': 1}}
   0  1  2
0  E  X  X
1  X  O  E
2  E  O  E
O
Y coordinate0
X coordinate0
O move
{0: {'X': 2, 'O': 1}, 1: {'X': 1, 'O': 1}, 2: {'X': 0, 'O': 1}}
   0  1  2
0  O  X  X
1  X  O  E
2  E  O  E
X
x
X move
{0: {'X': 2, 'O': 1}, 1: {'X': 1, 'O': 1}, 2: {'X': 1, 'O': 1}}
   0  1  2
0  O  X  X
1  X  O  E
2  E  O  X
O
Y coordinate1
X coordinate2
O move
{0: {'X'

This function lets one play two already trained models against each other

In [25]:
#function for testing 1 model against another

def test(board_size,model,model_2): #against random
    x_wins = 0
    o_wins = 0
    for i in range(100000):
#         print('Enter Board Size: ')
#         size = int(input())
        

        board = board_creator(board_size)
        
        horizontal_lists = {}
        vertical_lists = {}
        diag_1 = {}
        diag_2 = {}
        for i in range(board_size):
            horizontal_lists[i] = {'X':0, 'O':0}
            vertical_lists[i] = {'X': 0, 'O': 0}
        x_diag_1 = 0
        x_diag_2 = 0
        o_diag_1 = 0
        o_diag_2 = 0
        turns = board_size**2 #editted from just size, the input one
        tie_flag = True

        
        
        
        
        while(turns > 0): 
            
            #list of states seen in the episode, not the whole thing, used to go back and update rewards
            #print_board(board)
            
            who_win = 'E' 

            e_count = 0
            for i in board:
                for k in i:
                    if k=='E':
                        e_count+=1
            if e_count != turns:
                print('illegal game')
                
            p = get_turn(turns)
            #print(p)
            next_state, next_board, next_move = 0,0,0
            if p == 'X':
                next_state, next_board, next_move = choose_next_move(board,p,2,model)
                #print('x')
            else:
                next_state, next_board, next_move = choose_next_move(board,p,2,model_2)
            
        
    
            #Update win lists (which are used to check if a win has occured)
            x,y = next_move[0], next_move[1]

            horizontal_lists[x][p] += 1
            vertical_lists[y][p] += 1
            if (x == y):
                if (p =='X'):
                    x_diag_1 += 1
                else:
                    o_diag_1 += 1
            if ((x+y) == board_size-1):
                if (p =='X'):
                    x_diag_2 += 1
                else:
                    o_diag_2 += 1
                    
            #update board
            #print(p + ' move')
            #print(vertical_lists)
            board[y][x] = p
            e_count = 0
            for i in board:
                for k in i:
                    if k=='E':
                        e_count+=1
            if e_count != turns -1:
                print('illegal game')
            
            if (horizontal_lists[x][p] == board_size or x_diag_1 == board_size or x_diag_2 == board_size or o_diag_1 == board_size or o_diag_2 == board_size):
                #print(p + ' wins!') 
                #print_board(board)
                tie_flag = False
                who_win = p
                if who_win == 'X':
                    x_wins += 1
                else:
                    o_wins += 1
                break
            if (vertical_lists[y][p] == board_size):
                #print(p + ' wins!')
                #print_board(board) 
                tie_flag = False
                who_win = p
                if who_win == 'X':
                    x_wins += 1
                else:
                    o_wins += 1
                break
            turns -= 1
        #print_board(board)
    return x_wins,o_wins

In [27]:
test(3,x_states,o_states)

(100000, 0)